In [27]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
# Importing os and sys to define file path
import os
import sys

# Appending file path to sys path
module_path = os.path.abspath(os.path.join('code_files'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import CRUD module
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connection variables
username = "aacuser"
password = "deyocs340"
host = 'localhost'
port = 27017
database = 'aac'
collection = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Grazioso Salvare')

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
    # Company logo, header with unique identifier
    html.Div(id='hidden-div', style={'display':'none'}),
    
    html.Div(style= {'display': 'flex', 'justifyContent': 'space-between', 'alignItems': 'center', 'width': '100%'}, children= [
        html.Img(style={'width': '150px', 'height': 'auto'}, src='data:image/png;base64,{}'.format(encoded_image.decode())),
        html.Center(html.B(html.H1('CS-340 Dashboard - Daniel Leon')), style={'marginLeft': 'auto', 'marginRight': '500px'}),
        ]),
    
    html.Hr(),
    
    # Headings for widgets
    html.Div(style= {'display': 'flex', 'justifyContent': 'space-between', 'alignItems': 'center', 'width': '100%'}, children= [
        html.H4('Number of Rows'),
        html.H4('Filter by Rescue Type')
        ]),
    
    # Datatable widgets
    html.Div(style= {'display': 'flex', 'justifyContent': 'space-between', 'alignItems': 'center', 'width': '100%'}, children= [
        # Options for amount of datatable rows displayed
        html.Div(dcc.RadioItems(options= [{'label': '10', 'value': '10'}, {'label': '8', 'value': '8'}, 
                                          {'label': '6', 'value': '6'}, {'label': '4', 'value': '4'}], id= 'numRows')),

        # Options, values, and stying for dropdown menu
        html.Div(children=[dcc.Dropdown(options= [{'label':'Water Rescue', 'value':'water'}, 
                                                  {'label':'Mountain or Wilderness Rescue', 'value':'mountain'},
                                                  {'label':'Disaster Rescue or Individual Tracking', 'value':'disaster'}, 
                                                  {'label':'Reset Results', 'value':'reset'}], id= 'filter-type', placeholder= 'Rescue Type')], 
                 style={'width': '300px'})
        ]),
    html.Hr(),
    
    # Datatable layout and options
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single", # make rows selectable
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0], # select default to avoid error
        page_action="native",
        page_current=0,
        page_size=10 
    ),
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# This callback will filter the datatable based on a dropdown menu selection    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    # start case
    df = pd.DataFrame.from_records(db.read({}))
    
    # use dropdown menu to filter by rescue types
    if filter_type == 'water':
        df = pd.DataFrame.from_records(db.read({"animal_type":"Dog",
                         "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, 
                         "sex_upon_outcome":"Intact Female",
                         "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        
    elif filter_type == 'mountain':
        df = pd.DataFrame.from_records(db.read({"animal_type":"Dog",
                         "breed":{"$in":["German Shephard","Alaskan Malamute","Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, 
                         "sex_upon_outcome":"Intact Male",
                         "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(db.read({"animal_type":"Dog",
                         "breed":{"$in":["Doberman Pinscher","German Shephard","Golden Retriever", "Bloodhound", "Rottweiler"]}, 
                         "sex_upon_outcome":"Intact Male",
                         "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))
        
    elif filter_type == 'reset':
        df = pd.DataFrame.from_records(db.read({}))
        
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    return (data,columns)


# This callback will update the amount of datatable rows based on radio button selection
@app.callback(Output('datatable-id','page_size'),
              [Input('numRows', 'value')])
def update_rows(num_rows):
    
    # Start case
    page_size = 10
    
    if num_rows == '10':
        page_size = 10
    elif num_rows == '8':
        page_size = 8
    elif num_rows == '6':
        page_size = 6
    elif num_rows == '4':
        page_size = 4
    
    return page_size


# This callback will update the graph chart for the selected data entry
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    dff = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed', title='Preferred Animals')
        )    
    ]
    
    
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]




# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '800px', 'height': '450px'}, center=[30.75,-97.48], zoom=9, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

    
    

# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server(port=8060) 

Dash app running on https://strangebison-classicexhibit-3000.codio.io/proxy/8060/
